# RUSTY BARGAIN

## Descripción

Rusty Bargain es un servicio de venta de coches de segunda mano que está desarrollando una app para atraer a nuevos clientes. 

Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial, especificaciones técnicas, versiones de equipamiento y precios.

A Rusty Bargain le interesa:

* La calidad de la predicción
* La velocidad de la predicción
* El tiempo requerido para el entrenamiento

## Objetivo

Crear un modelo que determine el valor de mercado.

## Consideraciones

Para este proyecto consideraré:

1. Features:

- DateCrawled — fecha en la que se descargó el perfil de la base de datos
- VehicleType — tipo de carrocería del vehículo
- RegistrationYear — año de matriculación del vehículo
- Gearbox — tipo de caja de cambios
- Power — potencia (CV)
- Model — modelo del vehículo
- Mileage — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)
- RegistrationMonth — mes de matriculación del vehículo
- FuelType — tipo de combustible
- Brand — marca del vehículo
- NotRepaired — vehículo con o sin reparación
- DateCreated — fecha de creación del perfil
- NumberOfPictures — número de fotos del vehículo
- PostalCode — código postal del propietario del perfil (usuario)
- LastSeen — fecha de la última vez que el usuario estuvo activo


2. Target

- Price — precio (en euros)

## Carga de datos y librerias

In [1]:
# Cargaré todas las librerías que necesito.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from lightgbm.callback import early_stopping, log_evaluation
import math
import time
from sklearn.preprocessing import OrdinalEncoder

In [2]:
# Cargaré los datos del nuevo dataset.
car_data = pd.read_csv('car_data.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/brisna/github_projects/tripleten/rusty_bargain/car_data.csv'

In [ ]:
car_data

## Preparación de datos

In [ ]:
# Veré la información general de mi dataframe.
car_data.info()

In [ ]:
# Revisaré si hay datos nulos.
car_data.isna().sum()

Después de revisar la información puedo observar que hay algunas variables que requieren atención:
- Cambiaré los NaN de VehicleType, Gearbox, Model, Fueltype y NotRepaired por "Desconocido".

In [ ]:
# Cambiaré los NaN de VehicleType, Gearbox, Model, Fueltype y NotRepaired por "Desconocido".
unknown_columns = ["VehicleType", "Gearbox", "Model", "FuelType", "NotRepaired"]
car_data[unknown_columns] = car_data[unknown_columns].fillna("Desconocido")

In [ ]:
#Le daré formato a los encabezados para que queden uniformes.
new_col_names = []
for old_name in car_data.columns:
    name_stripped = old_name.strip()
    name_lowered = re.sub(r'(?<!^)(?=[A-Z])', '_', name_stripped).lower()
    name_no_spaces = name_lowered.replace(" ", "_")
    new_col_names.append(name_no_spaces)

car_data.columns = new_col_names

In [ ]:
# Revisaré cómo quedaron los encabezados.
car_data.sample(10)

In [ ]:
# Volveré a revisar mi DataFrame. 
car_data.info()

In [ ]:
# Observaré las estadísticas descriptivas de los datos.
car_data.describe()

Según la descripción de los datos, puedo observar que:
1. Hay 354,107 registros.
2. Se capturaron datos del 5 de marzo al 7 de abril de 2016.
3. El precio promedio es de $4,416.43 siendo 0 el precio más bajo y $20,000 el más alto registrado (revisaré esto más adelante).
4. La desviación estándar es de $4,514.34 lo que indica una alta variabilidad en los precios.
5. El año promedio de registro es 2004. Sin embargo, el año más bajo registrado es 1000 el más alto es 9999 por lo que revisaré también estos datos a detalle.
6. La potencia promedio es de 110.09 siendo 0 la potencia mínima y 20,000 la máxima lo que me indica que podría haber errores en los datos.
7. El kilometraje promedio es de 128,211.81km. El kilometraje más bajo es de 5,000km y el más alto es de 150,000km. La desviación estándar es de 37,906.59, lo que indica que los datos están muy dispersos.
8.  El mes promedio de registro es de 5.71, lo que indica que la mayoría de los vehículos se registraron entre mayo y junio. El mes mínimo registrado es 0 lo cual podría indicar que el mes de registro es desconocido. La desviación estándar es 3.73, lo que indica una distribución normal de los datos.
9. La fecha promedio de creación es el 20 de marzo de 2016 teniendo un periodo de tiempo de creación del 10 de marzo de 2014 al 7 de abril de 2016.
10.  Observo que la columna number_of_pictures contiene solo ceros como valor por lo que podría eliminarla.
11.  La columna postal_code tiene un rango que va del CP 1067 al 99998 lo que podría indicar que los autos pertenecen a diferentes regiones del país.

## Análisis exploratorio de datos

In [ ]:
car_data = car_data[car_data['price'] > 0]

In [ ]:
# Revisaré los datos atípicos relacionados con el precio sin considerar el precio cero (0).
plt.figure(figsize=(8, 5))
sns.boxplot(x=car_data['price'])
plt.title('Distribución de Precios de los Autos')
plt.show()

In [ ]:
# Calcularé el Q1 (percentil 25) y Q3 (percentil 75)
Q1 = car_data['price'].quantile(0.25)
Q3 = car_data['price'].quantile(0.75)

# Calcularé el IQR
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtraré los datos límites
car_data_filtered = car_data[(car_data['price'] >= lower_bound) & (car_data['price'] <= upper_bound)]

# Haré un boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(x=car_data_filtered['price'], showfliers=False)
plt.title('Distribución de Precios sin Valores Atípicos')
plt.show()

In [ ]:
# Quitaré las variables de poco valor previo a la realización de mi modelo.
data_model = car_data_filtered.drop(['postal_code', 'last_seen', 'date_created', 'date_crawled', 'number_of_pictures', 'registration_month'], axis=1)
data_model.describe()

In [ ]:
# Filtraré solo carros entre 1950 y 2025
data_model = data_model[(data_model['registration_year'] >= 1950) & (data_model['registration_year'] <= 2025)]

In [ ]:
# Veré la cantidad de autos que hay por año de registro.
registration_year_count = data_model['registration_year'].value_counts().sort_index()
plt.figure()
plt.bar(registration_year_count.index, registration_year_count.values)
plt.xlabel('Año de registro')
plt.ylabel('Cantidad de autos')
plt.title('Autos registrados por año')
plt.tight_layout()
plt.show()

In [ ]:
# Revisaré los datos atípicos relacionados con el power sin considerar el cero (0).
data_model = data_model[car_data['power'] > 0]

# Calcularé el Q1 (percentil 25) y Q3 (percentil 75)
Q1 = data_model['power'].quantile(0.25)
Q3 = data_model['power'].quantile(0.75)

# Calcularé el IQR
IQR = Q3 - Q1

lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

# Filtraré los datos límites
data_model_filtered = data_model[(data_model['power'] >= lower) & (data_model['power'] <= upper)].copy()

# Haré un boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(x=data_model_filtered['power'], showfliers=False)
plt.title('Distribución de Power sin Valores Atípicos')
plt.show() 

In [ ]:
data_model_filtered.reset_index(inplace=True)
data_model_filtered

In [ ]:
# Verificaré si tengo datos duplicados.
data_model_filtered[data_model_filtered.duplicated()]

In [ ]:
# Obtendré las variables categóricas para codificarlas.
categorical_chars = list(data_model_filtered.dtypes[data_model_filtered.dtypes == 'object'].index)
categorical_chars

In [ ]:
for char in categorical_chars:
    print(data_model_filtered[char].value_counts())
    print()

In [ ]:
# Haré una codificación ordinal para model y brand.
data_model_filtered[['model', 'brand']] = OrdinalEncoder().fit_transform(data_model_filtered[['model', 'brand']])
data_model_filtered

In [ ]:
data_ohe = pd.get_dummies(data_model_filtered, drop_first=True)
data_ohe

HALLAZGOS Y ACCIONES:

* El dataset tenía varias columnas con valores faltantes y outliers que podrían sesgar los modelos. Después de imputar valores nulos y filtrar outliers en RegistrationYear, Power y Mileage, los datos quedaron más homogéneos.

* Transformé las variables de fecha para extraer características (mes, año, etc.), lo que puede enriquecer los modelos basados en “edad” del anuncio o del vehículo.

* No encontré valores duplicados sospechosos en el dataset, y la columna NumberOfPictures estaba en 0, por lo que decidí descartarla.

## Entrenamiento de modelos

In [ ]:
# Definiré mis metricas para el modelo.
train_valid, test = train_test_split(data_ohe, test_size=0.2, random_state=12345)
train, valid = train_test_split(train_valid, test_size=0.25,random_state=12345)

In [ ]:
# Agregaré el RMSE para evaluar la precisión de mi modelo.
def rmse(prediction, real_value):
    return math.sqrt(mean_squared_error(prediction,real_value))

In [ ]:
# Preparo mis datos para entrenar mi modelo.
train_chars = train.drop('price', axis=1)
valid_chars = valid.drop('price', axis=1)
test_chars  = test.drop('price', axis=1)

train_target = train['price']
valid_target = valid['price']
test_target  = test['price']

print("train_chars:", train_chars.shape, "train_target:", train_target.shape)
print("valid_chars:", valid_chars.shape, "valid_target:", valid_target.shape)

# Entrenamiento y prueba combinando train y valid
X_train_lr = pd.concat([train_chars, valid_chars])
y_train = pd.concat([train_target, valid_target])

X_test_lr = test_chars
y_test = test_target


In [ ]:
# Haré un modelo de Regresión Lineal
model_lr = LinearRegression()
t0 = time.time()
model_lr.fit(X_train_lr, y_train)
train_preds_lr = model_lr.predict(X_train_lr)
test_preds_lr = model_lr.predict(X_test_lr)
results_lr = {'train_rmse': rmse(y_train, train_preds_lr), 'test_rmse': rmse(y_test, test_preds_lr), 'train_time': time.time() - t0}
print(f"Linear Regression -> Train RMSE: {results_lr['train_rmse']:.2f}, Test RMSE: {results_lr['test_rmse']:.2f}, Time: {results_lr['train_time']:.1f}s")

In [ ]:
# Haré un modelo de Random Forest
param_grid_rf = {'n_estimators': [100, 200], 'max_depth': [None, 10], 'min_samples_split': [2, 5]}
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
gs_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

t0 = time.time()
gs_rf.fit(X_train_lr, y_train)
model_rf = gs_rf.best_estimator_
train_preds_rf = model_rf.predict(X_train_lr)
test_preds_rf = model_rf.predict(X_test_lr)
results_rf = {'best_params': gs_rf.best_params_, 'train_rmse': rmse(y_train, train_preds_rf), 'test_rmse': rmse(y_test, test_preds_rf), 'train_time': time.time() - t0}
print(f"Random Forest -> Params: {results_rf['best_params']}, Train RMSE: {results_rf['train_rmse']:.2f}, Test RMSE: {results_rf['test_rmse']:.2f}, Time: {results_rf['train_time']:.1f}s")

In [ ]:
# Intentaré hacer un modelo de LightGBM.
model_lgbm = LGBMRegressor(
    n_estimators=1000,
    metric='rmse',
    verbose=-1
)
model_lgbm.fit(
    train_chars, train_target,
    eval_set=[(valid_chars, valid_target)],
    eval_metric='rmse',
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=50)
    ]
)
train_preds_lgbm = model_lgbm.predict(X_train_lr)
test_preds_lgbm = model_lgbm.predict(X_test_lr)
results_lgbm = {
    'train_rmse': rmse(y_train, train_preds_lgbm),
    'test_rmse': rmse(y_test, test_preds_lgbm),
    'train_time': time.time() - t0
}
print(f"LightGBM -> Train RMSE: {results_lgbm['train_rmse']:.2f}, Test RMSE: {results_lgbm['test_rmse']:.2f}, Time: {results_lgbm['train_time']:.1f}s")

Conclusiones después de entrenar los modelos:

* La Regresión Lineal es muy rápida, pero su calidad (RMSE) es la más baja.

* El modelo de Random Forest mejora mucho el RMSE, pero el tiempo de entrenamiento es moderado.

* LightGBM logra el mejor equilibrio entre calidad y velocidad (RMSE más bajo y entrenamiento más rápido que XGBoost o RF con muchos árboles).

## Conclusiones finales

Recomiendo usar LightGBM en producción ya que ofrece predicciones rápidas y de buena calidad aunque si el objetivo es maximizar la precisión y el tiempo de entrenamiento no es crítico, es buena idea también usar Random Forest con los parámetros óptimos encontrados.

La Regresión Lineal será una buena opción solo como referencia rápida cada vez que se modifiquen características o se añadan nuevas variables.